In [1]:
import json
import random

In [2]:
lang = "ja"
diff = "easy"
excl = None

vocabs_pool: set[str] = set()
exclusive_pool: set[str] = set()
alphabets: set[str] = set()

with open(rf"vocabs\{lang}\{lang}-{diff}.json") as file:
    json_data = json.load(file)
    for d in json_data:
        vocabs_pool.add(d)
        exclusive_pool.add(d)
        alphabets = alphabets.union(set(d))

if excl is not None:
    with open(rf"vocabs\{lang}\{lang}-{excl}.json") as file:
        json_data = json.load(file)
        for d in json_data:
            exclusive_pool.remove(d)

num_vocabs = len(vocabs_pool)
print(len(alphabets), "".join(sorted(alphabets)))
print(f"{len(vocabs_pool)=}")
print(f"{len(exclusive_pool)=}")

76 ァアィイウェエオカガキギクグケゲコゴサザシジスズセゼソゾタダチッツヅテデトドナニネノハバパヒビピフブプヘベホボポマミムメモャヤュユョヨラリルレロワヲンー
len(vocabs_pool)=1054
len(exclusive_pool)=1054


In [3]:
def change_character(vocab: str) -> set[str]:
    new_vocabs = set()
    for i, c in enumerate(vocab):
        for a in alphabets:
            if a != c:
                new_vocab = vocab[:i] + a + vocab[i + 1 :]
                new_vocabs.add(new_vocab)

    return new_vocabs


print(change_character("abc"))

{'abボ', 'abィ', 'abメ', 'abョ', 'aホc', 'abア', 'ヘbc', 'abゲ', 'aデc', 'aヘc', 'ゾbc', 'aダc', 'abネ', 'abギ', 'abセ', 'aゾc', 'リbc', 'aヤc', 'ヤbc', 'aヅc', 'abカ', 'スbc', 'aサc', 'メbc', 'abパ', 'abポ', 'aッc', 'abル', 'ニbc', 'aゲc', 'abズ', 'abゾ', 'abユ', 'ヲbc', 'ネbc', 'abデ', 'aリc', 'abノ', 'abッ', 'ジbc', 'aュc', 'チbc', 'aクc', 'abダ', 'abピ', 'セbc', 'abゴ', 'abソ', 'aプc', 'abモ', 'abロ', 'aルc', 'ィbc', 'abベ', 'abジ', 'aヲc', 'abム', 'abマ', 'aスc', 'ヨbc', 'abラ', 'aネc', 'aモc', 'サbc', 'ヒbc', 'aバc', 'エbc', 'aズc', 'aノc', 'aドc', 'ザbc', 'aポc', 'abヒ', 'abフ', 'abホ', 'abヅ', 'ウbc', 'ンbc', 'ハbc', 'aフc', 'abュ', 'abサ', 'イbc', 'aツc', 'タbc', 'ワbc', 'ノbc', 'ブbc', 'aトc', 'aヒc', 'モbc', 'ビbc', 'ロbc', 'abゼ', 'aガc', 'ソbc', 'ユbc', 'パbc', 'ナbc', 'aニc', 'ケbc', 'ツbc', 'abレ', 'abキ', 'abヨ', 'abハ', 'オbc', 'aハc', 'aケc', 'abバ', 'ェbc', 'aテc', 'abイ', 'ホbc', 'マbc', 'ヅbc', 'aワc', 'abビ', 'aユc', 'abコ', 'キbc', 'アbc', 'ュbc', 'abオ', 'aボc', 'aゼc', 'abウ', 'aエc', 'ラbc', 'aマc', 'aシc', 'ガbc', 'abタ', 'aジc', 'aイc', 'abガ', 'ギbc', 'aキc', 'aオc', 'aヨc', 'コbc', 'aコc', 'abテ'

In [4]:
def character_difference(from_: str, to: str) -> int:
    count = 0
    for c1, c2 in zip(from_, to):
        if c1 != c2:
            count += 1
    return count

In [5]:
def distance(
    from_: str, to: str, max_distance: int | None = None
) -> tuple[int, list[str]]:
    if len(from_) != len(to):
        return (-1, [])

    vocab_path = {from_: ""}
    queue = [(from_, 0)]
    while queue:
        (cur_vocab, cur_distance) = queue.pop(0)
        if cur_vocab == to:
            path = [cur_vocab]
            while path[0] != from_:
                path.insert(0, vocab_path[path[0]])
            return (cur_distance, path)

        if cur_distance == max_distance:
            continue

        for new_vocab in change_character(cur_vocab):
            if (new_vocab not in vocab_path) and (new_vocab in vocabs_pool):
                vocab_path[new_vocab] = cur_vocab
                queue.append((new_vocab, cur_distance + 1))

    return (-1, [])

In [6]:
print(distance("admin", "admit"))
print(distance("line", "firm"))
print(distance("センセイ", "モンダイ"))
print(distance("caro", "mano"))

(-1, [])
(-1, [])
(3, ['センセイ', 'ネンセイ', 'ネンダイ', 'モンダイ'])
(-1, [])


In [7]:
# easy: {"min_distance": 3, "max_distance": 4, "strict": False}
from collections import defaultdict

min_distance = 3
max_distance = 4
strict = False

starter = set()
involved = set()
counts = defaultdict(lambda: [0, 0, 0])  # vocab, pair, exclusive

pairs: list[tuple[str, str]] = []

for i, v1 in enumerate(vocabs_pool):
    for j, v2 in enumerate(vocabs_pool):
        dist = distance(v1, v2, max_distance=max_distance)
        if strict:
            difference = character_difference(v1, v2)
            flag = dist[0] > difference and dist[0] >= min_distance
        else:
            flag = dist[0] >= min_distance

        if flag:
            counts[len(v1)][1] += 1
            starter.add(v1)
            involved = involved.union(set(dist[1]))
            # print(dist)

            if any(v in exclusive_pool for v in dist[1]):
                counts[len(v1)][2] += 1
                pairs.append((v1, v2))

    counts[len(v1)][0] += 1
    if (i + 1) % (num_vocabs // 10) == 0:
        print(f'checking "{v1}"| {i+1}/{num_vocabs}')

checking "トイウヨリ"| 105/1054
checking "ギョウム"| 210/1054
checking "ツウカスル"| 315/1054
checking "トイアワセ"| 420/1054
checking "ショウセツ"| 525/1054
checking "ジッシスル"| 630/1054


In [ ]:
total_vocab = 0
total_combinations = 0
total_pairs = 0
total_exclusive = 0

print_table = []
for k, v in counts.items():
    count_vocab = v[0]
    vocab_combinations = count_vocab * (count_vocab - 1)
    count_pair = v[1]
    count_exlusive = v[2]

    cur_pair_rate = count_pair / vocab_combinations
    cur_exclusive_rate = count_exlusive / vocab_combinations
    to_print = [
        f"len {k}",
        f"{count_vocab}",
        f"{count_pair}/{vocab_combinations}",
        f"{cur_pair_rate:.2%}",
        f"{count_exlusive}/{vocab_combinations}",
        f"{cur_exclusive_rate:.2%}",
    ]
    print_table.append(to_print)

    total_vocab += count_vocab
    total_combinations += vocab_combinations
    total_pairs += count_pair
    total_exclusive += count_exlusive

pair_rate = total_pairs / total_combinations
exclusive_rate = total_exclusive / vocab_combinations
to_print = [
    f"total",
    f"{total_vocab}",
    f"{total_pairs}/{total_combinations}",
    f"{pair_rate:.2%}",
    f"{total_exclusive}/{total_combinations}",
    f"{exclusive_rate:.2%}",
]
print_table.append(to_print)

print(f"----- pair rate & exclusive rate -----")
for items in print_table:
    print(
        f"{items[0]:<5}, #{items[1]:5}:"
        f" {items[2]:>13} = {items[3]:<7} |"
        f" {items[4]:>13} = {items[5]:<7}"
    )
print("-" * 15)

starter_rate = len(starter) / num_vocabs
print(f"starter: {len(starter)}/{num_vocabs} = {starter_rate:.2%}")

participation_rate = len(involved) / num_vocabs
print(f"participation: {len(involved)}/{num_vocabs} = {participation_rate:.2%}")

----- pair rate & exclusive rate -----
len 4, #396  :    752/156420 = 0.48%   |    752/156420 = 0.48%  
len 5, #226  :     258/50850 = 0.51%   |     258/50850 = 0.51%  
total, #622  :   1010/207270 = 0.49%   |   1010/207270 = 1.99%  
---------------
starter: 173/622 = 27.81%
participation: 185/622 = 29.74%


In [ ]:
random.seed(0)
pairs_shuffled = pairs[:10000]
random.shuffle(pairs_shuffled)
with open(rf"vocabs\{lang}\{lang}-pairs-{diff}.json", "w") as f:
    json.dump(pairs_shuffled, f)

for i in range(5):
    pair = pairs_shuffled[i]
    dist = distance(pair[0], pair[1])
    print(dist)

(4, ['ケイヤク', 'ケイカク', 'セイカク', 'セイカツ', 'セイブツ'])
(4, ['カンジョウ', 'ゲンジョウ', 'ゲンショウ', 'インショウ', 'イッショウ'])
(3, ['コウゾウ', 'コウコウ', 'コウコク', 'コウモク'])
(3, ['ケッコン', 'ケッコウ', 'ケンコウ', 'シンコウ'])
(3, ['サイボウ', 'サイコウ', 'サンコウ', 'シンコウ'])


In [ ]:
def create_distance_graph():
    distance_graph: dict[tuple[str, str], tuple[int, list[str]]] = {}

    def calculate_distance(from_: str, to: str):
        if (from_, to) in distance_graph:
            return

        min_distance = -1
        min_path = []
        for new_vocab in change_character(from_):
            if new_vocab in vocabs_pool:
                calculate_distance(new_vocab, to)
                distance, path = distance_graph[(new_vocab, to)]
                if min_distance == -1 or 0 <= distance < min_distance:
                    min_distance = distance + 1
                    min_path = [from_] + path
        distance_graph[(from_, to)] = (min_distance, min_path)

    for v1 in vocabs_pool:
        for v2 in vocabs_pool:
            calculate_distance(v1, v2)
    return distance_graph


# distance_graph = create_distance_graph()
# print(distance_graph)